### Alex Khvatov Homework #4

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [6]:
#!wget https://archive.ics.uci.edu/static/public/222/bank+marketing.zip
#!unzip bank+marketing.zip 
#!unzip bank.zip
#!rm ./bank+marketing.zip
#!rm ./bank-names.txt
#!rm ./bank.*
#!rm ./bank-additional.zip

In [20]:
df = pd.read_csv("bank-full.csv", sep=";")
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [12]:
columns = ['age',
'job',
'marital',
'education',
'balance',
'housing',
'contact',
'day',
'month',
'duration',
'campaign',
'pdays',
'previous',
'poutcome',
'y']

In [21]:
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)
for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

#df.reset_index(inplace=True)

In [34]:
categorical_columns = [c for c in categorical_columns if c in columns]
categorical_columns.remove('y')
categorical_columns

['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']

In [27]:
numeric_columns = list(df.dtypes[df.dtypes != 'object'].index)
numeric_columns = [c for c in numeric_columns if c in columns]
numeric_columns

['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

In [29]:
#Encode target
df.y = (df.y == 'yes').astype(int)

In [36]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,0
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,0
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,0
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,0
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,0


In [17]:
df_full_train, df_test = train_test_split(df, test_size = 0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size = 0.25, random_state=1)


## Question 1

For each numerical variable, use it as score (aka prediction) and compute the AUC with the y variable as ground truth.

Use the training dataset for that

In [41]:
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve


In [45]:
#use age
y_pred = df_train['age']
y_pred

14137    56
33506    33
27524    32
34058    40
25922    54
         ..
22520    32
26678    52
4964     52
20083    32
7828     38
Name: age, Length: 27126, dtype: int64

In [46]:
y_val = df_train['y']

In [59]:
results = []

for num_col in numeric_columns:
    y_pred = df_train[num_col]
    if (roc_auc_score(y_val, y_pred) < 0.5):
        y_pred = -df_train[num_col]
        
    results.append((roc_auc_score(y_val, y_pred), num_col))    
    #print(f"{num_col} -- {}")
results.sort(reverse=True)
results

[(np.float64(0.8147002759670778), 'duration'),
 (np.float64(0.5985653242764153), 'previous'),
 (np.float64(0.5901276247352144), 'pdays'),
 (np.float64(0.5888313805382317), 'balance'),
 (np.float64(0.5714543015682159), 'campaign'),
 (np.float64(0.525957882383908), 'day'),
 (np.float64(0.512185717527344), 'age')]

In [ ]:
#answer 'duration'

## Question 2

Apply one-hot-encoding using DictVectorizer and train the logistic regression with these parameters:

LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)

What's the AUC of this model on the validation dataset? (round to 3 digits)

In [61]:
def train(df_train, y_train):
    dicts = df_train[categorical + numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train=dv.fit_transform(dicts)

    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)
    model.fit(X_train, y_train)
    return dv, model
    

In [64]:
y_train = df_train.y.values
dv, model = train(df_train, y_train)

NameError: name 'categorical' is not defined